In [1]:
%cd D:/uitds

D:\uitds


In [2]:
from collections import Counter

In [3]:
import json

# read json file
image_ova_file = "result_ova/Vintern_train_image_yes_no_ova_v1_epoch4.json"
text_ova_file = "result_ova/Vintern_train_text_ova_v1_epoch5.json"
yes_no_file = "result_ova/Vintern_train_yes_no_ova_v1_epoch2.json"
yes_no_2_file = "result_ova/Vintern_train_v4_2_epoch3.json"
multi_file = "result_ova/Vintern_train_multi_yes_no_ova_v1_epoch2.json"

best_result_file = "result_test_val/train_vintern_v6_1_epoch4.json" # 
best_result_2_file = "result_test_val/train_vintern_v1_epoch5.json"
best_result_3_file = "result_test_val/train_vintern_v1_1_epoch7.json" # many not 
best_result_4_file = "result_test_val/train_vintern_v3_epoch3.json" # many not 

with open(image_ova_file) as ova_image,\
        open(text_ova_file) as ova_text,\
        open(best_result_file) as best_result,\
        open(yes_no_file) as yes_no,\
        open(multi_file) as multi_ova,\
        open(best_result_2_file) as best_result_2,\
        open(yes_no_2_file) as yes_no_2,\
        open(best_result_3_file) as best_result_3,\
        open(best_result_4_file) as best_result_4:
    image_ova = json.load(ova_image)
    text_ova = json.load(ova_text)
    yes_no = json.load(yes_no)
    multi_ova = json.load(multi_ova)
    best_result = json.load(best_result)
    best_result_2 = json.load(best_result_2)
    best_result_3 = json.load(best_result_3)
    best_result_4 = json.load(best_result_4)
    yes_no_2 = json.load(yes_no_2)

In [4]:
# read txt file: data\train_val_group_split.txt
TXT_PATH = "data/train_val_group_split.txt"

def read_train_test_file(name_file):
    
    with open(name_file, 'r') as f:
        lines = f.readlines()
        num_train, num_val = int(lines[0].split()[0]), int(lines[0].split()[1])
        train_idx = list(map(int, lines[1].split()))
        val_idx = list(map(int, lines[2].split()))
    return num_train, num_val, train_idx, val_idx

num_train, num_val, train_idx, val_idx = read_train_test_file(TXT_PATH)

import json
JSON_PATH = "data/json/vimmsd-train.json"

json_file = open(JSON_PATH, 'r')
json_data = json.load(json_file)
labels = [json_data[str(i)]["label"] for i in val_idx]


In [5]:
import pandas as pd

df = pd.DataFrame({'image_ova': image_ova["results"].values(),\
    'text_ova': text_ova["results"].values(),\
    'yes_no':yes_no["results"].values(),\
    'best_result': best_result["results"].values(),\
    'best_result_2': best_result_2["results"].values(),\
    'best_result_3': best_result_3["results"].values(),\
    'best_result_4': best_result_4["results"].values(),\
    'multi_ova': multi_ova["results"].values(),\
    'yes_no_2': yes_no_2["results"].values(),\
    'label': labels})


In [6]:
def pair_count(*input_dicts):
    """
    Count pairs/groups from multiple input dictionaries
    Args:
        *input_dicts: Variable number of dictionaries, each with a 'results' key
    Returns:
        Dictionary with combined labels as keys and lists of indices as values
    """
    pair_best = {}
    
    # Get the number of results from the first dictionary
    n_results = len(input_dicts[0])
    
    for i in range(n_results):
        # Collect labels from all input dictionaries
        labels = []
        for input_dict in input_dicts:
            labels.append(input_dict[i])
            
        # Create key by joining all labels
        pair_key = " /".join(labels)
        
        # Add to pair_best dictionary
        if pair_key in pair_best:
            pair_best[pair_key].append(i)
        else:
            pair_best[pair_key] = [i]
            
    return pair_best

In [7]:
def pair_count_2(labels, *input_dicts):
    """
    Count pairs/groups from multiple input dictionaries
    Args:
        *input_dicts: Variable number of dictionaries, each with a 'results' key
    Returns:
        Dictionary with combined labels as keys and lists of indices as values
    """
    pair_best = {label:{} for label in labels}
    
    # Get the number of results from the first dictionary
    n_results = len(input_dicts[0])
    
    for i in range(n_results):
        # Collect labels from all input dictionaries
        pred_labels = []
        for input_dict in input_dicts:
            pred_labels.append(input_dict[i])
            
        # Create key by joining all labels
        pair_key= " ".join(pred_labels)
        
        # Add to pair_best dictionary
        if pair_key in pair_best[labels[i]]:
            pair_best[labels[i]][pair_key] +=1
        else:
            pair_best[labels[i]][pair_key] = 1
            
    return pair_best

In [8]:
# print dict in pretty format 

def print_dict(d):
    for k, v in d.items():
        print(k)
        for k2, v2 in v.items():
            print(f"    {k2}: {v2}")
        print()
        
print_dict(pair_count_2(df["label"],df["best_result"],df["best_result_2"],df["best_result_3"], df["yes_no"]))

multi-sarcasm
    multi-sarcasm multi-sarcasm multi-sarcasm Yes: 296
    not-sarcasm not-sarcasm not-sarcasm No: 65
    not-sarcasm not-sarcasm not-sarcasm Yes: 38
    multi-sarcasm multi-sarcasm not-sarcasm Yes: 15
    multi-sarcasm multi-sarcasm multi-sarcasm No: 13
    not-sarcasm multi-sarcasm multi-sarcasm Yes: 22
    multi-sarcasm not-sarcasm not-sarcasm Yes: 53
    not-sarcasm multi-sarcasm not-sarcasm No: 8
    not-sarcasm not-sarcasm multi-sarcasm No: 5
    multi-sarcasm not-sarcasm multi-sarcasm Yes: 21
    not-sarcasm multi-sarcasm multi-sarcasm No: 5
    multi-sarcasm multi-sarcasm image-sarcasm Yes: 2
    multi-sarcasm not-sarcasm not-sarcasm No: 8
    not-sarcasm image-sarcasm image-sarcasm Yes: 5
    multi-sarcasm image-sarcasm image-sarcasm Yes: 4
    not-sarcasm image-sarcasm multi-sarcasm Yes: 2
    multi-sarcasm image-sarcasm multi-sarcasm Yes: 4
    not-sarcasm multi-sarcasm image-sarcasm No: 1
    not-sarcasm not-sarcasm image-sarcasm Yes: 2
    not-sarcasm multi-s

In [9]:
[(i,len(v)) for (i,v) in pair_count(df["label"],df["best_result"],df["yes_no_2"], df["yes_no"]).items()]

[('multi-sarcasm /multi-sarcasm /Yes /Yes', 377),
 ('not-sarcasm /not-sarcasm /No /No', 598),
 ('multi-sarcasm /multi-sarcasm /No /Yes', 19),
 ('multi-sarcasm /not-sarcasm /No /No', 75),
 ('text-sarcasm /not-sarcasm /No /No', 13),
 ('image-sarcasm /not-sarcasm /Yes /Yes', 9),
 ('image-sarcasm /multi-sarcasm /Yes /Yes', 27),
 ('multi-sarcasm /not-sarcasm /Yes /Yes', 61),
 ('not-sarcasm /multi-sarcasm /Yes /Yes', 183),
 ('multi-sarcasm /multi-sarcasm /No /No', 10),
 ('not-sarcasm /not-sarcasm /Yes /Yes', 50),
 ('multi-sarcasm /not-sarcasm /No /Yes', 30),
 ('not-sarcasm /multi-sarcasm /No /No', 26),
 ('not-sarcasm /multi-sarcasm /Yes /No', 14),
 ('not-sarcasm /multi-sarcasm /No /Yes', 20),
 ('image-sarcasm /not-sarcasm /No /No', 6),
 ('not-sarcasm /not-sarcasm /No /Yes', 34),
 ('not-sarcasm /not-sarcasm /Yes /No', 19),
 ('text-sarcasm /not-sarcasm /No /Yes', 1),
 ('multi-sarcasm /not-sarcasm /Yes /No', 10),
 ('multi-sarcasm /multi-sarcasm /Yes /No', 15),
 ('image-sarcasm /not-sarcasm /No 

In [10]:
Counter(df["best_result"])

Counter({'not-sarcasm': 910, 'multi-sarcasm': 700, 'image-sarcasm': 9})

In [11]:
def old_transform_submit(best_result, yes_no):
    transformed_labels = []
    for i in range(len(best_result)):
        transformed_labels.append(best_result[i])
        if best_result[i] == "multi-sarcasm" and yes_no[i] == "No":
            transformed_labels[i] = "not-sarcasm"
        elif best_result[i] == "not-sarcasm" and yes_no[i] == "Yes":
            transformed_labels[i] = "multi-sarcasm"
    return transformed_labels

In [12]:
def transform_submit(best_result, best_result_2, yes_no):
    transformed_labels = []
    for i in range(len(best_result)):
        transformed_labels.append(best_result[i])
        if best_result[i] == "not-sarcasm" and best_result_2[i] == "not-sarcasm" and yes_no[i] == "Yes":
            transformed_labels[i] = "multi-sarcasm"
        elif best_result[i] == "not-sarcasm" and best_result_2[i] == "multi-sarcasm" and yes_no[i] == "Yes":
            transformed_labels[i] = "multi-sarcasm"
        elif best_result[i] == "multi-sarcasm" and best_result_2[i] == "multi-sarcasm" and yes_no[i] == "No":
            transformed_labels[i] = "not-sarcasm"
        elif best_result[i] == "not-sarcasm" and best_result_2[i] == "image-sarcasm" and yes_no[i] == "Yes":
            transformed_labels[i] = "image-sarcasm"
        elif best_result[i] == "multi-sarcasm" and best_result_2[i] == "image-sarcasm" and yes_no[i] == "Yes":
            transformed_labels[i] = "image-sarcasm"
    return transformed_labels

In [13]:
def transform_submit_2(best_result, best_result_2, best_result_3, yes_no):
    transformed_labels = []
    for i in range(len(best_result)):
        label1 = best_result[i]
        label2 = best_result_2[i]
        label3 = best_result_3[i]
        yes_no_label = yes_no[i]

        # Initialize the transformed label with the first result as default
        transformed_label = label1

        # Conditions for 'multi-sarcasm'
        if (
            # not-sarcasm not-sarcasm multi-sarcasm No: 6
            (label1 == 'not-sarcasm' and label2 == 'not-sarcasm' and label3 == 'multi-sarcasm' and yes_no_label == 'No') or
            # not-sarcasm multi-sarcasm not-sarcasm Yes: 18
            (label1 == 'not-sarcasm' and label2 == 'multi-sarcasm' and label3 == 'not-sarcasm' and yes_no_label == 'Yes') or
            # not-sarcasm image-sarcasm multi-sarcasm Yes: 2
            (label1 == 'not-sarcasm' and label2 == 'image-sarcasm' and label3 == 'multi-sarcasm' and yes_no_label == 'Yes') or
            # not-sarcasm not-sarcasm image-sarcasm Yes: 2
            (label1 == 'not-sarcasm' and label2 == 'not-sarcasm' and label3 == 'image-sarcasm' and yes_no_label == 'Yes') or
            # not-sarcasm not-sarcasm multi-sarcasm Yes: 3
            (label1 == 'not-sarcasm' and label2 == 'not-sarcasm' and label3 == 'multi-sarcasm' and yes_no_label == 'Yes') 
        ):
            transformed_label = 'multi-sarcasm'

        # Conditions for 'not-sarcasm'
        elif (
            # multi-sarcasm multi-sarcasm multi-sarcasm No: 19
            (label1 == 'multi-sarcasm' and label2 == 'multi-sarcasm' and label3 == 'multi-sarcasm' and yes_no_label == 'No') or
            # multi-sarcasm not-sarcasm not-sarcasm No: 17
            (label1 == 'multi-sarcasm' and label2 == 'not-sarcasm' and label3 == 'not-sarcasm' and yes_no_label == 'No') or
            # image-sarcasm multi-sarcasm multi-sarcasm Yes: 2
            (label1 == 'image-sarcasm' and label2 == 'multi-sarcasm' and label3 == 'multi-sarcasm' and yes_no_label == 'Yes')
        ):
            transformed_label = 'not-sarcasm'

        # Conditions for 'image-sarcasm'
        elif (
            # image-sarcasm image-sarcasm image-sarcasm Yes: 6
            (label1 == 'image-sarcasm' and label2 == 'image-sarcasm' and label3 == 'image-sarcasm' and yes_no_label == 'Yes') or
            # not-sarcasm image-sarcasm image-sarcasm Yes: 2
            (label1 == 'not-sarcasm' and label2 == 'image-sarcasm' and label3 == 'image-sarcasm' and yes_no_label == 'Yes')
        ):
            transformed_label = 'image-sarcasm'

        # Append the transformed label
        transformed_labels.append(transformed_label)

    return transformed_labels


In [14]:
from sklearn.metrics import f1_score

In [15]:
# print(f1_score(df["label"], transform_submit(df["best_result"], df["best_result_2"], df["best_result_3"],df["yes_no"]), average='macro'))
print(f1_score(df["label"], transform_submit_2(df["best_result"], df["best_result_2"],df["best_result_3"],df["yes_no"]), average='macro'))
print(f1_score(df["label"], old_transform_submit(df["best_result"], df["yes_no"]), average='macro'))
print(f1_score(df["label"], transform_submit(df["best_result"],df["best_result_2"], df["yes_no"]), average='macro'))
print(f1_score(df["label"], df["best_result"], average='macro'))

0.4181540236550133
0.39247712190526995
0.41185401882049344
0.38253904886744117


In [16]:
best_result_file , best_result_2_file, yes_no_file

('result_test_val/train_vintern_v6_1_epoch4.json',
 'result_test_val/train_vintern_v1_epoch5.json',
 'result_ova/Vintern_train_yes_no_ova_v1_epoch2.json')